# CAP5610 HW3 — Tree Ensembles & SHAP Study

This notebook mirrors Homework 3 and is self-contained: with the two datasets (`lncRNA_5_Cancers.csv` and `hw3-drug-screening-data.csv`) placed beside it, you can rerun every block to regenerate the figures, tables, and SHAP artefacts required in the assignment.

## Study Roadmap
1. **Shared setup** — import libraries, configure reproducible knobs, and define reusable helpers.
2. **Task 1** — explore the classification dataset and benchmark tree-based classifiers.
3. **Task 2** — interpret the best classifier with SHAP (per-cancer importances + patient force plots).
4. **Task 3** — profile the regression dataset and compare regressors on MAE/MSE/RMSE/R².
5. **Task 4** — run SHAP on the winning regressor for drug-specific insights and least-error explanation.
6. **Conclusion** — summarise artefacts and next steps.

## 0. Shared Setup & Reusable Utilities
The following cell loads all required libraries, defines runtime configuration (random seed, feature caps, SHAP sampling budgets), and introduces helper routines for logging and path resolution. Keeping the helpers here ensures the notebook is standalone.

In [2]:
# --- Imports & global configuration -------------------------------------------------
from pathlib import Path
import gc
import json
import math
import re
import time
import warnings
from typing import Dict, List, Optional, Tuple

import numpy as np
import pandas as pd
from IPython.display import display, HTML

from sklearn.compose import ColumnTransformer
from sklearn.ensemble import (
    GradientBoostingClassifier,
    GradientBoostingRegressor,
    RandomForestClassifier,
    RandomForestRegressor,
)
from sklearn.impute import SimpleImputer
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    f1_score,
    mean_absolute_error,
    mean_squared_error,
    r2_score,
)
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor

try:
    from xgboost import XGBClassifier, XGBRegressor
    HAVE_XGB = True
except Exception:
    HAVE_XGB = False

try:
    from lightgbm import LGBMClassifier, LGBMRegressor
    HAVE_LGBM = True
except Exception:
    HAVE_LGBM = False

try:
    from catboost import CatBoostClassifier, CatBoostRegressor
    HAVE_CAT = True
except Exception:
    HAVE_CAT = False

try:
    import shap
    HAVE_SHAP = True
except Exception:
    HAVE_SHAP = False

warnings.filterwarnings("ignore", category=UserWarning)

RANDOM_STATE = 42
CANCER_SET = {"KIRC", "LUAD", "LUSC", "PRAD", "THCA"}
PATIENT_ID_TO_PLOT = "TCGA-39-5011-01A"

CANCER_PRIMARY = Path("lncRNA_5_Cancers.csv")
CANCER_FALLBACK = Path("data/raw/lncRNA_5_Cancers.csv")
REG_PRIMARY = Path("GDSC2_13drugs.csv")
REG_FALLBACK = Path("data/raw/hw3-drug-screening-data.csv")

OUT_DIR = Path("hw3_outputs")
OUT_DIR.mkdir(parents=True, exist_ok=True)

MAX_FEATURES_CLASSIF = 1000
MAX_FEATURES_REGRESS = 1200
SHAP_SAMPLES_PER_CLASS = 30
SHAP_SAMPLES_REG = 100
BACKGROUND_SIZE = 200


def log(message: str) -> None:
    stamp = time.strftime("%H:%M:%S")
    print(f"[{stamp}] {message}")


def resolve_data_path(primary: Path, fallback: Optional[Path]) -> Optional[Path]:
    for candidate in (primary, fallback):
        if candidate is None:
            continue
        candidate_path = Path(candidate)
        if candidate_path.exists():
            return candidate_path
    return None


def select_top_variance_features(frame: pd.DataFrame, max_features: int) -> pd.DataFrame:
    if frame.shape[1] <= max_features:
        return frame
    variances = frame.var(axis=0, skipna=True)
    top_cols = variances.nlargest(max_features).index
    return frame.loc[:, top_cols]


def tree_background(sample: np.ndarray) -> np.ndarray:
    if sample.shape[0] <= BACKGROUND_SIZE:
        return sample
    rng = np.random.default_rng(RANDOM_STATE)
    idx = rng.choice(sample.shape[0], size=BACKGROUND_SIZE, replace=False)
    return sample[idx]


ModuleNotFoundError: No module named 'numpy'

## 1. Data Loading Utilities
These loaders detect identifier/target columns, enforce numeric typing, drop all-NaN features, and apply the variance cap defined above.

In [ ]:
def detect_id_and_target(df: pd.DataFrame) -> Tuple[Optional[str], Optional[str]]:
    id_col = None
    target_col = None
    for col in df.select_dtypes(include=["object"]).columns:
        if df[col].astype(str).str.contains(r"^TCGA-", na=False).any():
            id_col = col
            break
    if id_col is None:
        for col in df.columns:
            if re.search(r"(id|patient|sample)", col, re.I) and df[col].nunique(dropna=True) > 10:
                id_col = col
                break
    for col in df.columns:
        values = set(map(str, df[col].dropna().unique()))
        if values.issubset(CANCER_SET) and len(values) == len(CANCER_SET):
            target_col = col
            break
    if target_col is None:
        for col in df.columns:
            if re.search(r"(cancer|type|label|class)", col, re.I):
                target_col = col
                break
    return id_col, target_col


def memory_savvy_read_cancers(csv_path: Path, max_features: int) -> Tuple[pd.DataFrame, pd.Series, Optional[pd.Series], str, Optional[str]]:
    header_cols = pd.read_csv(csv_path, nrows=0).columns.tolist()
    sample_df = pd.read_csv(csv_path, nrows=200)
    id_col, target_col = detect_id_and_target(sample_df)
    if target_col is None:
        raise RuntimeError("Unable to detect target column in classification dataset.")

    feature_cols = [c for c in header_cols if c not in {id_col, target_col}]
    selected_cols = feature_cols[:max_features]
    usecols = [target_col] + ([id_col] if id_col else []) + selected_cols

    df = pd.read_csv(csv_path, usecols=usecols)
    y = df[target_col].astype(str)
    X = df.drop(columns=[target_col])

    ids = None
    if id_col and id_col in X.columns:
        ids = X[id_col].astype(str)
        X = X.drop(columns=[id_col])

    for col in X.columns:
        X[col] = pd.to_numeric(X[col], errors="coerce")
    X = X.astype(np.float32)
    X = X.loc[:, X.notna().any(axis=0)]
    X = select_top_variance_features(X, max_features)
    return X, y, ids, target_col, id_col


def memory_savvy_read_gdsc2(csv_path: Path, max_features: int) -> Tuple[pd.DataFrame, pd.Series, pd.Series, Dict[str, object]]:
    header_cols = pd.read_csv(csv_path, nrows=0).columns.tolist()
    target_col = "LN_IC50"

    id_cols: List[str] = []
    for cand in ["CELL_LINE_NAME", "cell_line", "CELL_LINE", "CellLine", "cellLine", "cell_line_name"]:
        if cand in header_cols:
            id_cols.append(cand)
            break
    for cand in ["DRUG_NAME", "drug_name", "Drug", "DRUG", "drug"]:
        if cand in header_cols:
            id_cols.append(cand)
            break

    if target_col not in header_cols:
        raise RuntimeError("Expected LN_IC50 column missing in regression dataset.")
    if not id_cols:
        raise RuntimeError("Could not detect cell line / drug identifier columns.")

    feature_cols = [c for c in header_cols if c not in id_cols + [target_col]]
    selected_cols = feature_cols[:max_features]
    usecols = id_cols + [target_col] + selected_cols

    df = pd.read_csv(csv_path, usecols=usecols)
    y = pd.to_numeric(df[target_col], errors="coerce").astype(np.float32)

    if len(id_cols) >= 2:
        keys = df[id_cols[0]].astype(str) + "|" + df[id_cols[1]].astype(str)
    else:
        keys = df[id_cols[0]].astype(str)

    X = df.drop(columns=id_cols + [target_col])
    for col in X.columns:
        X[col] = pd.to_numeric(X[col], errors="coerce")
    X = X.astype(np.float32)
    X = X.loc[:, X.notna().any(axis=0)]
    X = select_top_variance_features(X, max_features)
    meta = {
        "n_rows": len(df),
        "n_features": X.shape[1],
        "id_cols": id_cols,
        "target": target_col,
    }
    return X, y, keys, meta


## 2. Modelling & SHAP Utilities
These helpers encapsulate the repetitive parts of Tasks 1–4: training the model suites, logging metrics, and generating SHAP summaries. Artefacts are written to `hw3_outputs/` for direct inclusion in the report.

In [ ]:
def train_compare_classifiers(X: pd.DataFrame, y: pd.Series, random_state: int) -> Tuple[pd.DataFrame, Dict[str, Pipeline], Dict[int, str]]:
    class_names = sorted(y.astype(str).unique())
    class_to_idx = {c: i for i, c in enumerate(class_names)}
    idx_to_class = {i: c for c, i in class_to_idx.items()}
    y_encoded = y.map(class_to_idx).astype(int)

    X_train, X_test, y_train_enc, y_test_enc, y_train_lbl, y_test_lbl = train_test_split(
        X,
        y_encoded,
        y,
        test_size=0.2,
        random_state=random_state,
        stratify=y,
    )

    preprocess = ColumnTransformer([("num", SimpleImputer(strategy="median"), X.columns)], remainder="drop")

    models: Dict[str, Pipeline] = {}
    models["DecisionTree"] = Pipeline([
        ("prep", preprocess),
        ("clf", DecisionTreeClassifier(random_state=random_state, min_samples_leaf=2, class_weight="balanced")),
    ])
    models["RandomForest"] = Pipeline([
        ("prep", preprocess),
        ("clf", RandomForestClassifier(n_estimators=120, random_state=random_state, n_jobs=-1, class_weight="balanced_subsample")),
    ])
    models["GBM"] = Pipeline([
        ("prep", preprocess),
        ("clf", GradientBoostingClassifier(n_estimators=120, learning_rate=0.05, max_depth=3, random_state=random_state)),
    ])

    if HAVE_XGB:
        models["XGBoost"] = Pipeline([
            ("prep", preprocess),
            ("clf", XGBClassifier(
                objective="multi:softprob",
                eval_metric="mlogloss",
                n_estimators=160,
                learning_rate=0.05,
                max_depth=6,
                subsample=0.8,
                colsample_bytree=0.8,
                tree_method="hist",
                n_jobs=-1,
                random_state=random_state,
            )),
        ])
    if HAVE_LGBM:
        models["LightGBM"] = Pipeline([
            ("prep", preprocess),
            ("clf", LGBMClassifier(
                n_estimators=160,
                learning_rate=0.05,
                num_leaves=63,
                subsample=0.8,
                colsample_bytree=0.8,
                random_state=random_state,
                verbosity=-1,
            )),
        ])
    if HAVE_CAT:
        models["CatBoost"] = Pipeline([
            ("prep", preprocess),
            ("clf", CatBoostClassifier(iterations=160, learning_rate=0.05, depth=6, loss_function="MultiClass", random_seed=random_state, verbose=False)),
        ])

    rows = []
    fitted = {}
    for name, pipe in models.items():
        log(f"[Task 1] Training classifier: {name}")
        pipe.fit(X_train, y_train_enc)
        y_pred_enc = pipe.predict(X_test)
        y_pred = [idx_to_class[int(i)] for i in np.asarray(y_pred_enc, dtype=int)]
        acc = accuracy_score(y_test_lbl, y_pred)
        f1m = f1_score(y_test_lbl, y_pred, average="macro")
        rows.append({"Model": name, "Test_Accuracy": acc, "Test_F1_Macro": f1m})
        fitted[name] = pipe
        gc.collect()

    metrics = pd.DataFrame(rows).sort_values(by=["Test_F1_Macro", "Test_Accuracy"], ascending=False).reset_index(drop=True)

    best_name = metrics.iloc[0]["Model"]
    best_model = fitted[best_name]
    y_pred_best = [idx_to_class[int(i)] for i in np.asarray(best_model.predict(X_test), dtype=int)]
    cm = confusion_matrix(y_test_lbl, y_pred_best, labels=class_names)
    cm_df = pd.DataFrame(cm, index=[f"True_{c}" for c in class_names], columns=[f"Pred_{c}" for c in class_names])
    cm_df.to_csv(OUT_DIR / "task1_confusion_matrix.csv")

    report_df = pd.DataFrame(classification_report(
        y_test_lbl,
        y_pred_best,
        output_dict=True,
        zero_division=0,
        labels=class_names,
        target_names=class_names,
    )).T
    report_df.to_csv(OUT_DIR / "task1_classification_report.csv")

    metrics.to_csv(OUT_DIR / "task1_model_comparison.csv", index=False)
    (OUT_DIR / "task1_best_model.txt").write_text(str(best_name))
    return metrics, fitted, idx_to_class


def train_compare_regressors(X: pd.DataFrame, y: pd.Series, random_state: int) -> Tuple[pd.DataFrame, Dict[str, Pipeline]]:
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_state)
    preprocess = ColumnTransformer([("num", SimpleImputer(strategy="median"), X.columns)], remainder="drop")

    models: Dict[str, Pipeline] = {}
    models["DecisionTreeReg"] = Pipeline([
        ("prep", preprocess),
        ("reg", DecisionTreeRegressor(random_state=random_state, min_samples_leaf=2)),
    ])
    models["RandomForestReg"] = Pipeline([
        ("prep", preprocess),
        ("reg", RandomForestRegressor(n_estimators=120, random_state=random_state, n_jobs=-1)),
    ])
    models["GBMReg"] = Pipeline([
        ("prep", preprocess),
        ("reg", GradientBoostingRegressor(n_estimators=120, learning_rate=0.05, max_depth=3, random_state=random_state)),
    ])

    if HAVE_XGB:
        models["XGBReg"] = Pipeline([
            ("prep", preprocess),
            ("reg", XGBRegressor(
                n_estimators=160,
                learning_rate=0.05,
                max_depth=6,
                subsample=0.8,
                colsample_bytree=0.8,
                tree_method="hist",
                n_jobs=-1,
                random_state=random_state,
            )),
        ])
    if HAVE_LGBM:
        models["LGBMReg"] = Pipeline([
            ("prep", preprocess),
            ("reg", LGBMRegressor(
                n_estimators=160,
                learning_rate=0.05,
                num_leaves=63,
                subsample=0.8,
                colsample_bytree=0.8,
                random_state=random_state,
                verbosity=-1,
            )),
        ])
    if HAVE_CAT:
        models["CatBoostReg"] = Pipeline([
            ("prep", preprocess),
            ("reg", CatBoostRegressor(iterations=160, learning_rate=0.05, depth=6, loss_function="RMSE", random_seed=random_state, verbose=False)),
        ])

    rows = []
    fitted = {}
    for name, pipe in models.items():
        log(f"[Task 3] Training regressor: {name}")
        pipe.fit(X_train, y_train)
        preds = pipe.predict(X_test)
        mae = mean_absolute_error(y_test, preds)
        mse = mean_squared_error(y_test, preds)
        rmse = math.sqrt(mse)
        r2 = r2_score(y_test, preds)
        rows.append({"Model": name, "MAE": mae, "MSE": mse, "RMSE": rmse, "R2": r2})
        fitted[name] = pipe
        gc.collect()

    metrics = pd.DataFrame(rows).sort_values(by=["RMSE", "MAE"], ascending=[True, True]).reset_index(drop=True)
    metrics.to_csv(OUT_DIR / "task3_regressor_comparison.csv", index=False)
    (OUT_DIR / "task3_best_model.txt").write_text(str(metrics.iloc[0]["Model"]))
    return metrics, fitted


def shap_task2(best_model: Pipeline, X: pd.DataFrame, y: pd.Series, sample_ids: Optional[pd.Series], patient_id: str, idx_to_class: Dict[int, str]) -> None:
    if not HAVE_SHAP:
        log("[Task 2] SHAP package not installed — skipping interpretability step.")
        return

    estimator = best_model.named_steps[list(best_model.named_steps.keys())[-1]]
    explainer = shap.TreeExplainer(estimator, feature_perturbation="tree_path_dependent")

    df_full = X.copy()
    df_full["__y__"] = y.values
    idxs = []
    rng = np.random.default_rng(RANDOM_STATE)
    for cancer in sorted(CANCER_SET):
        class_idx = df_full.index[df_full["__y__"] == cancer].tolist()
        if len(class_idx) > SHAP_SAMPLES_PER_CLASS:
            class_idx = list(rng.choice(class_idx, SHAP_SAMPLES_PER_CLASS, replace=False))
        idxs.extend(class_idx)
    idxs = sorted(set(idxs))
    X_shap = X.loc[idxs]

    shap_raw = explainer.shap_values(X_shap)
    if isinstance(shap_raw, list):
        shap_by_class = shap_raw
    else:
        shap_arr = np.asarray(shap_raw)
        shap_by_class = [shap_arr[:, :, i] for i in range(shap_arr.shape[2])] if shap_arr.ndim == 3 else [shap_arr]

    records = []
    classes = list(estimator.classes_) if hasattr(estimator, "classes_") else list(range(len(shap_by_class)))
    for i, raw_cls in enumerate(classes):
        display_name = idx_to_class.get(int(raw_cls), str(raw_cls))
        mean_abs = np.abs(shap_by_class[i]).mean(axis=0)
        top_idx = np.argsort(mean_abs)[::-1][:10]
        for rank, feat_idx in enumerate(top_idx, start=1):
            records.append(
                {
                    "CancerType": display_name,
                    "Rank": rank,
                    "Feature": X_shap.columns[feat_idx],
                    "Mean|SHAP|": float(np.asarray(mean_abs[feat_idx]).reshape(-1)[0]),
                }
            )
    pd.DataFrame(records).to_csv(OUT_DIR / "task2a_top10_features_per_cancer.csv", index=False)

    shap.initjs()
    if sample_ids is not None and sample_ids.notna().any():
        matches = sample_ids[sample_ids.astype(str) == patient_id]
        idx = matches.index[0] if not matches.empty else X.index[0]
    else:
        idx = X.index[0]
    row = X.loc[[idx]]
    shap_row_raw = explainer.shap_values(row)

    if isinstance(shap_row_raw, list):
        shap_rows = shap_row_raw
    else:
        shap_arr = np.asarray(shap_row_raw)
        if shap_arr.ndim == 3:
            shap_rows = [shap_arr[:, :, i] for i in range(shap_arr.shape[2])]
        elif shap_arr.ndim == 2:
            shap_rows = [shap_arr]
        else:
            shap_rows = [shap_arr.reshape(shap_arr.shape[0], -1)]

    expected_values = np.asarray(explainer.expected_value)
    inline_visuals = []
    for i, raw_cls in enumerate(classes):
        display_name = idx_to_class.get(int(raw_cls), str(raw_cls))
        expected = expected_values[i] if expected_values.ndim > 0 else expected_values
        force_plot = shap.force_plot(expected, shap_rows[i][0, :], row, feature_names=row.columns, matplotlib=False)
        shap.save_html(str(OUT_DIR / f"task2b_forceplot_{display_name}_patient_{patient_id.replace(':','-')}.html"), force_plot)
        inline_visuals.append((display_name, force_plot))

    for display_name, force_plot in inline_visuals:
        log(f"Task 2b force plot for {display_name}")
        display(HTML(force_plot.html()))


def shap_task4(best_reg_model: Pipeline, X: pd.DataFrame, y: pd.Series, keys: pd.Series) -> None:
    if not HAVE_SHAP:
        log("[Task 4] SHAP package not installed — skipping interpretability step.")
        return

    estimator = best_reg_model.named_steps[list(best_reg_model.named_steps.keys())[-1]]
    explainer = shap.TreeExplainer(estimator, feature_perturbation="tree_path_dependent")

    if keys.str.contains(r"\|").any():
        drug_names = keys.str.split("|").str[1]
    else:
        drug_names = pd.Series(["ALL"] * len(keys), index=keys.index)

    rng = np.random.default_rng(RANDOM_STATE)
    sample_idx = rng.choice(X.index, size=min(SHAP_SAMPLES_REG, len(X)), replace=False)
    X_shap = X.loc[sample_idx]
    drugs_shap = drug_names.loc[sample_idx]

    shap_vals = explainer.shap_values(X_shap)
    shap_arr = np.asarray(shap_vals)
    if shap_arr.ndim != 2:
        raise ValueError("Unexpected SHAP output shape for regression.")

    records = []
    for drug in sorted(drugs_shap.unique()):
        mask = (drugs_shap == drug).values
        if mask.sum() == 0:
            continue
        mean_abs = np.abs(shap_arr[mask]).mean(axis=0)
        top_idx = np.argsort(mean_abs)[::-1][:10]
        for rank, feat_idx in enumerate(top_idx, start=1):
            records.append(
                {
                    "Drug": drug,
                    "Rank": rank,
                    "Feature": X_shap.columns[feat_idx],
                    "Mean|SHAP|": float(np.asarray(mean_abs[feat_idx]).reshape(-1)[0]),
                }
            )
    pd.DataFrame(records).to_csv(OUT_DIR / "task4a_top10_features_per_drug.csv", index=False)

    preds = best_reg_model.predict(X)
    errors = np.abs(preds - y.values)
    idx_min = int(np.argmin(errors))
    least_key = keys.iloc[idx_min]
    row = X.iloc[[idx_min]]
    shap_row = explainer.shap_values(row)[0, :]
    mean_abs = np.abs(shap_row)
    top_idx = np.argsort(mean_abs)[::-1][:10]
    pd.DataFrame(
        {
            "Rank": np.arange(1, 11),
            "Feature": X.columns[top_idx],
            "Absolute_SHAP": mean_abs[top_idx].astype(float),
        }
    ).to_csv(OUT_DIR / f"task4b_top10_features_least_error_{least_key.replace('|', '_')}.csv", index=False)


## 3. Task 1 — Classification Dataset Reconnaissance
We load the lncRNA expression matrix with the memory-savvy routine. The summary table captures the sample size, retained feature count, and identifier columns for citation in the written report.

In [ ]:
cancer_path = resolve_data_path(CANCER_PRIMARY, CANCER_FALLBACK)
assert cancer_path is not None, "Classification CSV missing – please place lncRNA_5_Cancers.csv alongside the notebook."

Xc, yc, sample_ids, class_col, id_col = memory_savvy_read_cancers(cancer_path, MAX_FEATURES_CLASSIF)

summary_cls = pd.DataFrame(
    {
        "rows": [len(Xc)],
        "selected_features": [Xc.shape[1]],
        "target_column": [class_col],
        "id_column": [id_col],
    }
)

log("Task 1 dataset loaded.")
display(summary_cls)
Xc.iloc[:5, :10]


## 4. Task 1 — Model Comparison
We benchmark the required classifiers under a shared preprocessing pipeline (median imputation). Macro-F1 is the primary ranking metric to respect class balance across the five cancers.

In [ ]:
cls_results, cls_models, idx_to_class = train_compare_classifiers(Xc, yc, RANDOM_STATE)

log("Task 1 model sweep complete.")
cls_results


## 5. Task 1 — Confusion Matrix & Per-Class Report
The confusion matrix and class-wise precision/recall/F1 are required in the homework write-up. They are exported to `hw3_outputs/` and displayed here for quick inspection.

In [ ]:
confusion = pd.read_csv(OUT_DIR / "task1_confusion_matrix.csv", index_col=0)
classification_report_df = pd.read_csv(OUT_DIR / "task1_classification_report.csv", index_col=0)

log("Task 1 evaluation artefacts loaded from hw3_outputs/.")
display(confusion)
classification_report_df


## 6. Task 2 — SHAP on Best Classifier
The top-ranked classifier becomes the subject of SHAP analysis. We compute per-cancer top-10 genes and generate patient-level force plots for `TCGA-39-5011-01A` as mandated.

In [ ]:
best_classifier_name = cls_results.iloc[0]["Model"]
best_classifier = cls_models[best_classifier_name]

log(f"Task 2 interprets the {best_classifier_name}.")
shap_task2(best_classifier, Xc, yc, sample_ids, PATIENT_ID_TO_PLOT, idx_to_class)

task2_top = pd.read_csv(OUT_DIR / "task2a_top10_features_per_cancer.csv")
log("Task 2 SHAP tables saved to hw3_outputs/.")
task2_top.head(15)


The five force plots above are also saved as HTML files in `hw3_outputs/task2b_forceplot_<Cancer>_patient_TCGA-39-5011-01A.html` for sharing or screenshot capture.

## 7. Task 3 — Regression Dataset Reconnaissance
We repeat the data audit for the GDSC2 drug-response table, capturing dimensionality and ID columns to justify preprocessing choices.

In [ ]:
regression_path = resolve_data_path(REG_PRIMARY, REG_FALLBACK)
assert regression_path is not None, "Regression CSV missing – please place hw3-drug-screening-data.csv (renamed to GDSC2_13drugs.csv) alongside the notebook."

Xr, yr, keys, meta = memory_savvy_read_gdsc2(regression_path, MAX_FEATURES_REGRESS)

summary_reg = pd.DataFrame(
    {
        "rows": [meta["n_rows"]],
        "selected_features": [meta["n_features"]],
        "target_column": [meta["target"]],
        "id_columns": [" & ".join(meta["id_cols"])],
    }
)

log("Task 3 dataset loaded.")
display(summary_reg)
Xr.iloc[:5, :10]


## 8. Task 3 — Regressor Comparison
With preprocessing aligned to the classification case, we evaluate the regression ensemble and rank models using RMSE (primary) plus MAE/MSE/R².

In [ ]:
reg_results, reg_models = train_compare_regressors(Xr, yr, RANDOM_STATE)

log("Task 3 model sweep complete.")
reg_results


## 9. Task 4 — SHAP on Best Regressor
We apply SHAP to the top regressor to fulfil Tasks 4a and 4b: per-drug feature rankings and the least-error drug–cell-line explanation.

In [ ]:
best_regressor_name = reg_results.iloc[0]["Model"]
best_regressor = reg_models[best_regressor_name]

log(f"Task 4 interprets the {best_regressor_name}.")
shap_task4(best_regressor, Xr, yr, keys)

per_drug = pd.read_csv(OUT_DIR / "task4a_top10_features_per_drug.csv")
least_error_path = sorted(OUT_DIR.glob("task4b_top10_features_least_error_*.csv"))[-1]
least_error = pd.read_csv(least_error_path)

(per_drug.head(20), least_error)


## 10. Conclusion & Artefact Checklist
- All tables/figures required by the assignment are in `hw3_outputs/`.
- Rerun with different feature caps or SHAP sample sizes by adjusting the configuration cell at the top.
- A natural extension is hyper-parameter tuning around the winning models or annotating the highlighted genes/drugs with biological context.